In [1]:
# Basis
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from math import floor, sqrt, pi
from statistics import mean
from glob import glob
import random
import warnings; warnings.filterwarnings('ignore')

# Geo
import geopandas as gp
from geopandas import GeoDataFrame, GeoSeries, sjoin
import shapely
from shapely import wkt
from shapely.geometry import Point, Polygon, MultiPolygon, LineString
import shapely.speedups; shapely.speedups.enable()

# Plot
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors
import matplotlib.animation as animation
import matplotlib.font_manager as font_manager
osfont = font_manager.FontProperties(family='Open Sans', style='normal', size=60)

In [2]:
# Definieer projecties
crs = {'init': 'epsg:4326'}          # Standaard WGS84
amersfoort = {'init': 'epsg:28992'}  # Rijkscoördinatendriehoek
cea = {'proj':'cea'}                 # Equal area

In [3]:
# Oppervlaktes van bomen (m²)
kleine_boom = 10.5
middel_boom = 14.3
grote_boom = 18.7

# Oppervlakte van een microbosje (4 bomen)
tiny_forest_area = grote_boom * 4  # m²
# tiny_forest_area = 280  # m²

# Minimale afstand voor nieuwe bomen
min_dist = 2.44  # m; dit is de minimale afstand tussen grote bomen
gem_afstand_tot_rand = 0  # m; dit is de minimale afstand tot muren

In [88]:
def to_crs(gdf, crs_out=amersfoort):
    '''Returns a GeoDataFrame with coordinates projected as specified (default epsg:28992)'''
    gdf = gdf.to_crs(crs_out)  # Use geopandas built-in method
    gdf.crs = crs_out  # Set CRS property as well, just to be sure
    return gdf

def random_points_within(poly, k, min_dist=min_dist, gem_afstand_tot_rand=gem_afstand_tot_rand):
    '''Returns a list of lists of k random Point objects that are within poly, seperated by at least min_dist'''
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    i = 0
    while len(points) < k:
        i += 1
        # Pak een random punt op de x-as, en vind en een random punt op de y-as dat in het polygoon ligt
        try:
            x = random.uniform(min_x, max_x)
            x_line = LineString([(x, min_y), (x, max_y)])
            x_line_intercept_min, x_line_intercept_max = x_line.intersection(poly).xy[1].tolist()
            y = random.uniform(x_line_intercept_min, x_line_intercept_max)
        except:
            continue
        point = Point([x, y])
        # Controleer of de afstand van een nieuwe boom tot bestaande bomen kleiner is dan min_dist
        # (Dit werkt nog niet goed wanneer min_dist te groot is)
        dist = []
        gdf = GeoDataFrame(points, crs=cea)
        for ind in range(len(gdf)):
            dist.append(gdf.iat[ind, 0].distance(point) < min_dist)  # bool
        # Voeg de nieuwe boom toe, behalve als er een andere nieuwe boom dichterbij ligt dan min_dist
        try:
            if sum(dist) == 0 and poly.exterior.distance(point) < gem_afstand_tot_rand:
                points.append(point)
            elif i > k * 10:
                return points
            else:
                continue
        except:
            if sum(dist) == 0:
                points.append(point)
            elif i > k * 10:
                return points
            else:
                continue
    return points

def random_points(row, colname='nieuwe_bomen', min_dist=min_dist):
    '''Returns a list of lists of random points, 
    based on a geometry and a value of nieuwe_bomen in a GeoDataFrame'''
    points = random_points_within(row['geometry'], row[colname], min_dist=min_dist)
    return points

def plant_trees(gdf, colname='nieuwe_bomen', min_dist=min_dist):
    '''Returns a GeoSeries (with epsg:28992) of random points within each geometry of a GeoDataFrame'''
    # change gdf.crs to cea for equal area calculations 
    # get a list of lists of random points within gdf.geometry
    # transform into a Series, then a GeoSeries, and change crs to epsg:28992
    return to_crs(GeoSeries(pd.Series(to_crs(gdf, cea).apply(
        random_points, colname=colname, min_dist=min_dist, axis=1)).apply(
        lambda x: Series(x)).stack().tolist(), crs=cea), amersfoort)


# Importeren

In [89]:
gdf = pd.read_csv('final/gdf_final.csv', index_col=0)
gdf = GeoDataFrame(gdf, crs=amersfoort, geometry=gdf.geometry.apply(wkt.loads))

# Voor buurten

In [90]:
gdf_all = gdf.copy()
del gdf

potential_trees_b = gdf_all[['Buurt', 'ruimte', 'nieuwe_bomen']].groupby(
    'Buurt').sum().sort_values('nieuwe_bomen', ascending=False).head(10).tail(5)
potential_trees_b['potential_trees'] = np.nan

for buurt in potential_trees_b.index.tolist():
    print(f'{buurt}: {potential_trees_b.at[buurt, "nieuwe_bomen"]}')
    gdf = gdf_all[gdf_all.Buurt == buurt].copy()
    potential_trees_b.at[buurt, 'potential_trees'] = len(plant_trees(gdf, min_dist=min_dist))

potential_trees_b.reset_index().to_csv('potential_trees_b.csv', index=False)

gdf = gdf_all.copy()
del gdf_all

Teleport: 2210
Zeeburgereiland Noordoost: 2189


IndexError: list index out of range

# Voor stadsdelen

In [ ]:
gdf_all = gdf.copy()
del gdf

potential_trees_sd = gdf_all[['Stadsdeel', 'ruimte', 'nieuwe_bomen']].groupby('Stadsdeel').sum()
potential_trees_sd['potential_trees'] = np.nan

for stadsdeel in potential_trees_sd.index.tolist():
    if stadsdeel == 'Westpoort':
        potential_trees_sd.at[stadsdeel, 'potential_trees'] = 0
    else:
        gdf = gdf_all[gdf_all.Stadsdeel == stadsdeel].copy()
        potential_trees_sd.at[stadsdeel, 'potential_trees'] = len(plant_trees(gdf, min_dist=min_dist))

potential_trees_sd.reset_index().to_csv('potential_trees_sd.csv', index=False)

gdf = gdf_all.copy()
del gdf_all